In [1]:
import pandas as pd
import json
import psycopg2

#기존에 뽑은 연관규칙으로 돌려보기

In [5]:
assoc_rule = pd.read_pickle("/content/drive/MyDrive/yeoreodigm/data_files/10.10_association_rule.pkl")
assoc_rule.head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,[검멀레해변],[성산일출봉(UNESCO 세계자연유산)],0.080268,0.343893,0.042148,0.525084,1.526882,0.014544,1.381522
4,[검멀레해변],[우도(해양도립공원)],0.080268,0.306309,0.055570,0.692308,2.260163,0.030984,2.254497


In [78]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_attraction.csv",index_col=0)
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...


In [6]:
course = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/08.21_course.csv",index_col=0)
course.head(2)

,id,places,theme,cluster,score
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",10,167322
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",7,163116


In [8]:
course['places'] = course['places'].apply(lambda x: x[1:-1].split(','))

In [10]:
course['places'] = course['places'].apply(lambda x : list(map(int,x)))

In [77]:
def parse_id_to_title(x):
  title = attraction[attraction['id']== x ]['title'].values[0]
  return title

print(list(map(parse_id_to_title,[602,  10,  58,  29,])))    

TypeError: ignored

In [79]:
def parse_title_to_id(x):
  id = attraction[attraction['title']==x]['id'].values[0]
  return id

print(list(map(parse_title_to_id,['수목원테마공원(야시장길&LED공원)', '카멜리아힐', '성읍민속마을', '아쿠아플라넷 제주'])))    

[602, 10, 58, 29]


In [16]:
assoc_rule['test'] = assoc_rule['antecedents'].apply(lambda x: list(map(parse_title_to_id,x)))

In [19]:
assoc_rule.sample(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,test
3410,"[광치기해변, 섭지코지]","[성산일출봉(UNESCO 세계자연유산), 협재해수욕장]",0.029530,0.172349,0.010738,0.363636,2.109884,0.005649,1.300594,"[52, 9]"
10586,"[섭지코지, 카멜리아힐, 협재해수욕장]","[월정리해변, 우도(해양도립공원)]",0.054228,0.116779,0.017718,0.326733,2.797883,0.011385,1.311844,"[9, 10, 8]"


In [24]:
assoc_rule.rename(columns={'test':'antecedents_id'},inplace=True)

In [26]:
assoc_rule['consequents_id'] = assoc_rule['consequents'].apply(lambda x: list(map(parse_title_to_id,x)))

In [28]:
print(list(map(parse_title_to_id,['오설록티뮤지엄', '월정리해변', '성산일출봉(UNESCO 세계자연유산)'])))    

[22, 13, 3]


In [50]:
assoc_rule.shape

(10629, 11)

###id를 가지고 연관규칙 돌리기

In [60]:
entire_basket = []
from tqdm import tqdm
for input in tqdm(course['places'].values):
  same_basket = []
  for val,attr in zip(assoc_rule['antecedents_id'].values,assoc_rule['consequents_id'].values):
    match_rate = len(set(input).intersection(set(val))) /len(val)
    if match_rate >=1.0 :
      same_basket += list(attr)
  same_basket = set(list(same_basket))
  recommend = list(same_basket - set(input)) #이미 일정에 있는애들은 연관규칙 기반 추천에서 제외
  entire_basket.append(recommend)

100%|██████████| 3725/3725 [01:01<00:00, 60.79it/s]


In [61]:
course['recommend'] = entire_basket

In [62]:
course[course['id']==2614]

,id,places,theme,cluster,score,recommend
2638,2614,"[3, 5, 134, 10, 78, 18, 22, 310]","[0.125, 0.0, 0.375, 0.5, 0.125, 0.125, 0.125, ...",5,332540,"[2, 8, 9, 13, 15, 16, 17, 24, 58, 59]"


In [80]:
course['place_title'] = course['places'].apply(lambda x: list(map(parse_id_to_title,x)))

In [81]:
course[course['id']==2614]

,id,places,theme,cluster,score,recommend,place_title
2638,2614,"[3, 5, 134, 10, 78, 18, 22, 310]","[0.125, 0.0, 0.375, 0.5, 0.125, 0.125, 0.125, ...",5,332540,"[2, 8, 9, 13, 15, 16, 17, 24, 58, 59]","[성산일출봉(UNESCO 세계자연유산), 천지연폭포, 검멀레해변, 카멜리아힐, 제주..."


###title을 가지고 연관규칙 돌리기

In [ ]:
entire_basket = []
cnt = 0
from tqdm import tqdm
for input in tqdm(course['place_title'].values):
  if cnt ==2638:
    print('input : ',input)
  same_basket = []
  for val,attr in zip(assoc_rule['antecedents'].values,assoc_rule['consequents'].values):
    match_rate = len(set(input).intersection(set(val))) /len(val)
    if match_rate >=1.0 :
      if cnt == 2638:
        print('교집합',set(input).intersection(set(val)))
        print(list(val), "\t\t",list(attr))
      same_basket += list(attr)
  same_basket = set(list(same_basket))
  recommend = list(same_basket - set(input)) #이미 일정에 있는애들은 연관규칙 기반 추천에서 제외
  entire_basket.append(recommend)
  cnt+=1

In [83]:
course['recommend_title'] = entire_basket

###결과 값 분포 확인

In [116]:
for i in course.sample(50)['recommend_title'].values:
  print(i[::-1])

['천지연폭포', '카멜리아힐', '애월해안도로', '섭지코지', '김녕해수욕장', '쇠소깍', '만장굴', '성산일출봉(UNESCO 세계자연유산)', '비자림', '오설록티뮤지엄']
['천지연폭포', '월정리해변', '카멜리아힐', '섭지코지', '쇠소깍', '오설록티뮤지엄', '우도(해양도립공원)']
['천지연폭포', '주상절리대(중문대포해안)', '김녕해수욕장', '쇠소깍', '이호테우해수욕장', '우도(해양도립공원)']
['천지연폭포', '월정리해변', '주상절리대(중문대포해안)', '섭지코지', '협재해수욕장', '성산일출봉(UNESCO 세계자연유산)', '쇠소깍']
['정방폭포', '천지연폭포', '카멜리아힐', '주상절리대(중문대포해안)', '월정리해변', '애월해안도로', '아쿠아플라넷 제주', '만장굴', '협재해수욕장', '비자림', '오설록티뮤지엄', '곽지해수욕장', '우도(해양도립공원)']
['하고수동해변', '월정리해변', '섭지코지', '협재해수욕장', '성산일출봉(UNESCO 세계자연유산)', '쇠소깍']
['천지연폭포', '섭지코지', '에코랜드 테마파크', '성산일출봉(UNESCO 세계자연유산)', '협재해수욕장', '김녕해수욕장', '쇠소깍', '성읍민속마을', '이호테우해수욕장', '우도(해양도립공원)']
['정방폭포', '카멜리아힐', '주상절리대(중문대포해안)', '월정리해변', '섭지코지', '에코랜드 테마파크', '만장굴', '비자림', '오설록티뮤지엄', '성읍민속마을', '용머리해안', '우도(해양도립공원)']
['정방폭포', '천지연폭포', '용두암', '주상절리대(중문대포해안)', '애월해안도로', '협재해수욕장', '오설록티뮤지엄', '쇠소깍', '용머리해안', '우도(해양도립공원)']
['천지연폭포', '섭지코지', '월정리해변', '주상절리대(중문대포해안)', '애월해안도로', '성산일출봉(UNESCO 세계자연유산)', '쇠소깍', '천제연폭포', '우도(해양도립공원)']
['천지연폭포', '월정리해변',

####두 방식 결과가 일치하는지 비교

In [85]:
tmp = [2, 8, 9, 13, 15, 16, 17, 24, 58, 59]
list(map(parse_id_to_title,tmp)) # 우도(해양도립공원),용머리해안,성읍민속마을,쇠소깍,협재해수욕장,섭지코지,월정리해변,주상절리대(중문대포해안),애월해안도로,정방폭포

['우도(해양도립공원)',
 '협재해수욕장',
 '섭지코지',
 '월정리해변',
 '쇠소깍',
 '주상절리대(중문대포해안)',
 '정방폭포',
 '용머리해안',
 '성읍민속마을',
 '애월해안도로']

##함수로 만들기

In [104]:
def recommend_by_assoc_rule(input,association_rule):
  recommend_list = []
  for antecedents,consequents in zip(association_rule['antecedents_id'].values,association_rule['consequents_id'].values):
    match_rate = len(set(input).intersection(set(antecedents))) / len(antecedents)
    if match_rate >=1.0 : #cause를 가진다면
      #print(consequents)
      recommend_list += consequents
      
  print(set(recommend_list))
  recommend_list = list(set(recommend_list) - set(input))
  
  return recommend_list

In [106]:
recommend_by_assoc_rule([3,5,134,10,78,18,22,310],assoc_rule)

{2, 3, 5, 134, 8, 9, 13, 15, 16, 17, 18, 310, 22, 24, 58, 59}


[2, 8, 9, 13, 15, 16, 17, 24, 58, 59]

#연관규칙 재설정 (상위 여행지에만 쏠려있으므로)


In [118]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [120]:
course['place_title']
te = TransactionEncoder()
test_array = te.fit(course['place_title'].values).transform(course['place_title'].values)

In [122]:
df = pd.DataFrame(test_array,columns = te.columns_)
frequent_itemsets = apriori(df,min_support=0.01,use_colnames=True)

In [123]:
frequent_itemsets

,support,itemsets
0,0.020671,(1100고지(휴게소))
1,0.018523,(거문오름(UNESCO 세계자연유산))
2,0.014765,(건강과성박물관)
3,0.080268,(검멀레해변)
4,0.012886,(곶자왈)
...,...,...
3844,0.013960,"(우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 섭지코..."
3845,0.010201,"(우도(해양도립공원), 성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 섭지코..."
3846,0.011544,"(쇠소깍, 오설록티뮤지엄, 성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 애..."
3847,0.010201,"(쇠소깍, 오설록티뮤지엄, 성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 카..."


In [135]:
new_rule = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.1)

In [136]:
new_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(곽지해수욕장),(검멀레해변),0.109799,0.080268,0.015302,0.139364,1.736228,0.006489,1.068665
1,(검멀레해변),(곽지해수욕장),0.080268,0.109799,0.015302,0.190635,1.736228,0.006489,1.099877
2,(김녕해수욕장),(검멀레해변),0.101208,0.080268,0.014765,0.145889,1.817508,0.006641,1.076829
3,(검멀레해변),(김녕해수욕장),0.080268,0.101208,0.014765,0.183946,1.817508,0.006641,1.101388
4,(검멀레해변),(만장굴),0.080268,0.103624,0.010201,0.127090,1.226454,0.001884,1.026883
...,...,...,...,...,...,...,...,...,...
28178,"(우도(해양도립공원), 카멜리아힐)","(성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 월정리해변, 오설록티뮤지엄)",0.102819,0.037852,0.011812,0.114883,3.035017,0.007920,1.087028
28179,"(우도(해양도립공원), 월정리해변)","(성산일출봉(UNESCO 세계자연유산), 협재해수욕장, 카멜리아힐, 오설록티뮤지엄)",0.116779,0.036242,0.011812,0.101149,2.790975,0.007580,1.072212
28180,"(월정리해변, 오설록티뮤지엄)","(성산일출봉(UNESCO 세계자연유산), 우도(해양도립공원), 협재해수욕장, 카멜리아힐)",0.102819,0.035436,0.011812,0.114883,3.241950,0.008169,1.089758
28181,"(성산일출봉(UNESCO 세계자연유산), 카멜리아힐)","(협재해수욕장, 우도(해양도립공원), 월정리해변, 오설록티뮤지엄)",0.095302,0.035436,0.011812,0.123944,3.497653,0.008435,1.101029


In [137]:
new_rule.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,28183.000000,28183.000000,28183.000000,28183.000000,28183.000000,28183.000000,28183.000000
mean,0.071191,0.158944,0.017057,0.325664,2.390895,0.008273,1.363437
std,0.056584,0.108685,0.010706,0.187430,1.010420,0.004567,0.543178
min,0.011007,0.012886,0.010201,0.100000,0.628389,-0.011113,0.785875
25%,0.033020,0.072752,0.011544,0.171053,1.666077,0.005785,1.099448
50%,0.055839,0.118926,0.013691,0.273128,2.195789,0.007729,1.193321
75%,0.095302,0.263624,0.018255,0.447059,2.882225,0.009816,1.399973
max,0.424430,0.424430,0.188725,0.959184,13.226449,0.086249,16.074631


In [138]:
new_rule = new_rule[new_rule['lift']>1.5]

In [139]:
new_rule.shape

(23546, 9)

In [140]:
new_rule.sample(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17863,"(정방폭포, 쇠소깍)","(성산일출봉(UNESCO 세계자연유산), 섭지코지, 만장굴)",0.061745,0.042685,0.011007,0.178261,4.176237,0.008371,1.164987
14203,"(애월해안도로, 주상절리대(중문대포해안))","(성산일출봉(UNESCO 세계자연유산), 천지연폭포)",0.038389,0.109530,0.011544,0.300699,2.745355,0.007339,1.273372


###타이틀 기반 연관규칙에서 id 값 추가하기

In [142]:
new_rule.sample(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_id
10408,(외돌개),"(협재해수욕장, 섭지코지, 성산일출봉(UNESCO 세계자연유산))",0.064966,0.101745,0.015034,0.231405,2.274363,0.008424,1.168697,[49]
3587,(천제연폭포),"(섭지코지, 쇠소깍)",0.067114,0.117315,0.014497,0.216000,1.841190,0.006623,1.125873,[20]


In [141]:
new_rule['antecedents_id'] = new_rule['antecedents'].apply(lambda x: list(map(parse_title_to_id,x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [144]:
new_rule['consequents_id'] = new_rule['consequents'].apply(lambda x: list(map(parse_title_to_id,x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####새롭게 적용한 연관규칙 결과 확인

In [145]:
new_result = recommend_by_assoc_rule(input = [3,5,134,10,78,18,22,310],association_rule=new_rule)

{1, 2, 3, 4, 5, 134, 6, 8, 9, 10, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 28, 29, 31, 37, 174, 49, 179, 310, 58, 59, 61, 76, 78, 336, 80, 83, 219, 223}


In [146]:
new_result

[1,
 2,
 4,
 6,
 8,
 9,
 13,
 15,
 16,
 17,
 19,
 20,
 21,
 24,
 25,
 28,
 29,
 31,
 37,
 174,
 49,
 179,
 58,
 59,
 61,
 76,
 336,
 80,
 83,
 219,
 223]

####파일로 저장

In [148]:
new_rule['antecedents'] = new_rule['antecedents'].apply(lambda x: list(x))
new_rule['consequents'] = new_rule['consequents'].apply(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [149]:
new_rule.head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_id,consequents_id
0,[곽지해수욕장],[검멀레해변],0.109799,0.080268,0.015302,0.139364,1.736228,0.006489,1.068665,[31],[134]
1,[검멀레해변],[곽지해수욕장],0.080268,0.109799,0.015302,0.190635,1.736228,0.006489,1.099877,[134],[31]


####저장 전 전처리

In [182]:
new_rule['antecedents_str'] = new_rule['antecedents'].apply(lambda x: ','.join(x))
new_rule['consequents_str'] = new_rule['consequents'].apply(lambda x: ','.join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [184]:
new_rule.sample(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_id,consequents_id,antecedents_str,consequents_str
16964,"[에코랜드 테마파크, 카멜리아힐, 오설록티뮤지엄]",[월정리해변],0.026309,0.283490,0.014228,0.540816,1.907709,0.006770,1.560400,"[18, 10, 22]",[13],"에코랜드 테마파크,카멜리아힐,오설록티뮤지엄",월정리해변
20373,"[우도(해양도립공원), 에코랜드 테마파크, 섭지코지]","[성산일출봉(UNESCO 세계자연유산), 협재해수욕장]",0.026309,0.172349,0.010470,0.397959,2.309031,0.005936,1.374742,"[2, 18, 9]","[3, 8]","우도(해양도립공원),에코랜드 테마파크,섭지코지","성산일출봉(UNESCO 세계자연유산),협재해수욕장"


In [185]:
new_rule.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'antecedents_id', 'consequents_id', 'antecedents_str',
       'consequents_str'],
      dtype='object')

In [186]:
new_rule = new_rule[['antecedents_id', 'consequents_id', 'antecedents_str',
       'consequents_str' ,'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', ]]

In [187]:
new_rule.sample(3)

,antecedents_id,consequents_id,antecedents_str,consequents_str,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3403,"[3, 9]",[13],"성산일출봉(UNESCO 세계자연유산),섭지코지",월정리해변,0.188725,0.283490,0.083490,0.442390,1.560513,0.029988,1.284966
24871,"[3, 13, 10]","[8, 59]","성산일출봉(UNESCO 세계자연유산),월정리해변,카멜리아힐","협재해수욕장,애월해안도로",0.049933,0.111678,0.013960,0.279570,2.503360,0.008383,1.233044
26270,"[3, 5, 22]","[9, 15, 59]","성산일출봉(UNESCO 세계자연유산),천지연폭포,오설록티뮤지엄","섭지코지,쇠소깍,애월해안도로",0.050201,0.040000,0.011275,0.224599,5.614973,0.009267,1.238069


In [189]:
new_rule.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/10.11_association_rule.csv")

####저장 잘 됐는지 로드해서 확인

In [192]:
new_rule_check = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/10.11_association_rule.csv",index_col=0)

In [193]:
new_rule_check.sample(3)

,antecedents_id,consequents_id,antecedents_str,consequents_str,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11242,"[9, 22, 223]",[15],"섭지코지,오설록티뮤지엄,이호테우해수욕장",쇠소깍,0.023893,0.263624,0.012081,0.505618,1.917950,0.005782,1.489487
17088,"[8, 13, 22]",[2],"협재해수욕장,월정리해변,오설록티뮤지엄",우도(해양도립공원),0.072752,0.306309,0.035436,0.487085,1.590176,0.013152,1.352448
26302,"[3, 9, 15, 59]","[8, 22]","성산일출봉(UNESCO 세계자연유산),섭지코지,쇠소깍,애월해안도로","협재해수욕장,오설록티뮤지엄",0.029530,0.172081,0.014228,0.481818,2.799957,0.009147,1.597739


##연관규칙 DB 반영

In [199]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [202]:
db = load_db()
cursor = db.cursor()

In [205]:
db.rollback()

In [194]:
new_rule.head(2)

,antecedents_id,consequents_id,antecedents_str,consequents_str,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,[31],[134],곽지해수욕장,검멀레해변,0.109799,0.080268,0.015302,0.139364,1.736228,0.006489,1.068665
1,[134],[31],검멀레해변,곽지해수욕장,0.080268,0.109799,0.015302,0.190635,1.736228,0.006489,1.099877


#####db 호환을 위해 numpy64 -> int로 전환..

In [206]:
new_rule['antecedents_id'] = new_rule['antecedents_id'].apply(lambda x: list(map(int,x))) 
new_rule['consequents_id'] = new_rule['consequents_id'].apply(lambda x: list(map(int,x))) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [208]:
for rule in tqdm(new_rule.values):
  antecedents = rule[0]
  consequents = rule[1]
  cursor.execute("insert into association_rule(antecedents, consequents) values(%s, %s)",(antecedents,consequents))
db.commit()

100%|██████████| 23546/23546 [54:21<00:00,  7.22it/s]
